In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pytube import YouTube

from zipfile import ZipFile
from urllib.request import urlretrieve

from IPython.display import Image

from collections import deque

def bfs_vectorized(img, pos, labels, count):
    rows, cols = img.shape
    queue = [pos]
    total_pixel = 0
    visited = np.zeros_like(img, dtype=bool)
    
    while queue:
        x, y = queue.pop(0)
        if visited[x, y]:
            continue
        visited[x, y] = True
        labels[x, y] = count
        total_pixel += 1
        
        # Tạo mặt nạ cho các pixel xung quanh chưa được kiểm tra và có cùng giá trị
        mask = (np.abs(np.roll(img, 1, axis=0) - img) == 0) | \
               (np.abs(np.roll(img, -1, axis=0) - img) == 0) | \
               (np.abs(np.roll(img, 1, axis=1) - img) == 0) | \
               (np.abs(np.roll(img, -1, axis=1) - img) == 0)
        
        # Cập nhật hàng đợi bằng các pixel mới thỏa mãn điều kiện
        neighbors = np.argwhere(mask & ~visited)
        for nx, ny in neighbors:
            queue.append((nx, ny))
            
    return total_pixel

def bfs(img, pos, labels, count):
    queue = deque([pos])
    labels[pos] = count
    total_pixel = 0
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    while queue:
        x, y = queue.popleft()
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(img) and 0 <= ny < len(img[0]) and img[x,y] == img[nx,ny] and labels[nx,ny] == 0:
                labels[nx,ny] = count
                queue.append((nx, ny))
                total_pixel +=1

    return total_pixel

def bfs_object_detection(binary_frame, min_objPixel = 10, max_distance=50):
    rows, cols = binary_frame.shape
    labels = np.zeros((rows,cols), dtype=int)
    
    count = 1
    object = []

    for x in range(rows):
        for y in range(cols):
            if binary_frame[x,y] > 0 and labels[x,y] == 0:
                pixels = bfs(binary_frame, (x,y), labels, count)    
                if pixels >= min_objPixel:
                    xs, ys = np.where(labels == count)
                    if len(xs) > 0 and len(ys) > 0:
                        min_x, max_x = np.min(xs), np.max(xs)
                        min_y, max_y = np.min(ys), np.max(ys)
                        object.append((min_x, min_y, max_x, max_y,pixels))
                count+=1

    return object

# def colision(obj1,obj2):
#     min_x1, min_y1, max_x1, max_y1 = obj1
#     min_x2, min_y2, max_x2, max_y2 = obj2
#     return not (max_x1 < min_x2 or min_x1 > max_x2 or max_y1 < min_y2 or min_y1 > max_y2)


def tracking_colision(curr_area,pre_obj, tolerance_pixel=0):
    for i in range(len(pre_obj)):
        for j in range(i+1, len(pre_obj)):
            pre_area1 = pre_obj[i][4]
            pre_area2 = pre_obj[j][4]
            total_preArea = pre_area1 + pre_area2
            if total_preArea - tolerance_pixel <= curr_area <= total_preArea + tolerance_pixel:
                return True
    return False

def bfs_obj_collision(video_path,min_objPixel = 10, debug = False):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Không thể mở video.")
        exit()
    
    if debug:
        output_folder = 'frames_output'
        os.makedirs(output_folder, exist_ok=True)

    frame_count = 0
    previous_objects = []
    while True:
        ret, frame = cap.read()
        current_time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

        if not ret:
            print("Hết video")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray_frame, (5, 5), 0)
        _, binary = cv2.threshold(blurred, 147, 255, cv2.THRESH_BINARY_INV )

        #Tìm đối tượng
        objects = bfs_object_detection(binary, min_objPixel)
        
        #có sự thay đổi thì tracking
        if len(objects) < len(previous_objects):
            for i in range(len(objects)):
                curr_area = objects[i][4]
                if tracking_colision(curr_area,previous_objects):
                    print(f"Chạm nhau tại: {current_time_ms}, Frame: {frame_count}")
                    break
        
        previous_objects = objects
            
        if debug:
            debug_frame = frame.copy()
            for obj in objects:
                min_x, min_y, max_x, max_y, area = obj
                cv2.rectangle(debug_frame, (min_y, min_x), (max_y, max_x), (0, 255, 0), 2)

            # Lưu khung hình có bounding boxes
            frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.png')
            cv2.imwrite(frame_filename, debug_frame)
            print(f"Frame {frame_count}: {len(objects)}")

        frame_count+=1
        if frame_count == 56:
            break


In [2]:
bfs_obj_collision('videoplayback.mp4',10,True)

Frame 0: 22
Frame 1: 22
Frame 2: 22
Frame 3: 22
Frame 4: 22
Frame 5: 22
Frame 6: 22
Frame 7: 22
Frame 8: 22
Frame 9: 22
Frame 10: 22
Frame 11: 22
Frame 12: 22
Frame 13: 23
Frame 14: 23
Frame 15: 23
Frame 16: 23
Frame 17: 23
Frame 18: 23
Frame 19: 23
Frame 20: 23
Frame 21: 23
Frame 22: 23
Frame 23: 23
Frame 24: 23
Frame 25: 23
Frame 26: 23
Frame 27: 23
Frame 28: 23
Frame 29: 23
Frame 30: 23
Frame 31: 23
Frame 32: 23
Frame 33: 23
Frame 34: 23
Frame 35: 23
Frame 36: 23
Frame 37: 23
Frame 38: 23
Frame 39: 23
Frame 40: 23
Frame 41: 23
Frame 42: 23
Frame 43: 23
Frame 44: 23
Frame 45: 23
Frame 46: 23
Frame 47: 23
Frame 48: 23
Chạm nhau tại: 1960.0, Frame: 49
Frame 49: 22
Frame 50: 23
Frame 51: 23
Frame 52: 23
Frame 53: 23
Frame 54: 23
Frame 55: 23
